In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Polygon, Point
import numpy as np

from typing import List
import os
from pathlib import Path

from nuplan.database.nuplan_db_orm.nuplandb import LidarBox
import pyarrow as pa
import numpy as np
from tqdm import tqdm

from asim.common.geometry.base import StateSE3
from asim.common.geometry.bounding_box.bounding_box import BoundingBoxSE3
from asim.common.geometry.constants import DEFAULT_ROLL, DEFAULT_PITCH

In [ ]:
from asim.dataset.observation.agent_datatypes import BoundingBoxType


name_mapping = {
    "vehicle": BoundingBoxType.VEHICLE,
    "bicycle": BoundingBoxType.BICYCLE,
    "pedestrian": BoundingBoxType.PEDESTRIAN,
    "traffic_cone": BoundingBoxType.TRAFFIC_CONE,
    "barrier": BoundingBoxType.BARRIER,
    "czone_sign": BoundingBoxType.CZONE_SIGN,
    "generic_object": BoundingBoxType.GENERIC_OBJECT,
}

In [ ]:
# log_name = "2021.06.07.12.54.00_veh-35_01843_02314"
log_name = "2021.07.16.18.06.21_veh-38_04933_05307"
# log_name = "2021.05.12.22.28.35_veh-35_00620_01164"
# log_name = "2021.07.16.20.45.29_veh-35_00600_01084"

# log_name = "2021.08.17.16.57.11_veh-08_01200_01636"
# log_name = "2021.08.24.13.12.55_veh-45_00386_00472"

with pa.OSFile(f"/home/daniel/asim_workspace/nuplan_logs/{log_name}.arrow", "rb") as source:
    reader = pa.RecordBatchFileReader(source)
    read_table = reader.read_all()

num_iterations = len(read_table)

In [ ]:
from asim.dataset.maps.gpkg.gpkg_map import GPKGMap

# name = "nuplan_us-pa-pittsburgh-hazelwood"
name = "nuplan_us-nv-las-vegas-strip"
# name = "carla_town03"
# name = "carla_town05"
map_file_path = Path(f"/home/daniel/asim_workspace/asim/notebooks/{name}.gpkg")

map_api = GPKGMap(map_file_path)
map_api.initialize()

In [ ]:
from asim.dataset.maps.map_datatypes import MapSurfaceType

map_api._gpd_dataframes[MapSurfaceType.WALKWAY]

In [ ]:
from typing import Optional, Tuple
from asim.common.geometry.base import Point2D
from asim.common.geometry.tranform_2d import translate_along_yaw
from asim.common.visualization.maps.plot_maps import _plot_map_on_ax_v2
from asim.common.visualization.color.lib.ellis import ELLIS_5


def load_bounding_boxes(idx: int) -> Tuple[List[BoundingBoxSE3], BoundingBoxSE3]:
    bounding_boxes = []
    for pa_array in read_table["bb_frame"][idx]:
        bounding_boxes.append(BoundingBoxSE3.from_array(pa.array(pa_array).to_numpy()))

    ego_bounding_box = BoundingBoxSE3.from_array(pa.array(read_table["bb_ego"][idx]).to_numpy())
    return bounding_boxes, ego_bounding_box


def plot_bounding_box(ax, box: BoundingBoxSE3, color: str = "red", alpha: float = 1.0):
    box_polygon = box.shapely_polygon
    x, y = box_polygon.exterior.xy
    ax.fill(x, y, color=color, alpha=alpha, edgecolor="black", linewidth=2, zorder=3)
    arrow = np.zeros((2, 2))
    arrow[0] = box.center.state_se2.point_2d.array
    arrow[1] = translate_along_yaw(box.center.state_se2, Point2D(box.length / 2.0 + 0.5, 0.0)).point_2d.array
    ax.plot(arrow[:, 0], arrow[:, 1], color="black", alpha=alpha, linewidth=2, zorder=3)


def plot_frame(ax: Optional[plt.Axes] = None, idx: int = 0):

    radius = 100

    bounding_boxes, ego_bounding_box = load_bounding_boxes(idx)

    point_2d = ego_bounding_box.center.state_se2.point_2d

    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))
    _plot_map_on_ax_v2(ax, map_api, point_2d, radius=radius)

    for box_idx, box in enumerate(bounding_boxes):
        plot_bounding_box(ax, box, color=ELLIS_5[4].hex, alpha=1.0)

    plot_bounding_box(ax, ego_bounding_box, color=ELLIS_5[0].hex, alpha=1.0)

    buffer_radius = 0.8 * radius
    ax.set_xlim(point_2d.x - buffer_radius, point_2d.x + buffer_radius)
    ax.set_ylim(point_2d.y - buffer_radius, point_2d.y + buffer_radius)

    ax.set_aspect("equal", adjustable="box")

    return ax


plot_frame(idx=0)

In [ ]:
import io

from PIL import Image


def create_animation_gif(start_idx=0, end_idx=num_iterations, step=10, fps=20):
    # Calculate duration per frame in milliseconds
    duration = int(1000 / fps)

    # Create frames
    images = []
    for i in tqdm(range(start_idx, end_idx, step)):
        fig, ax = plt.subplots(figsize=(10, 10))
        plot_frame(ax, i)

        # Creating PIL image from fig
        buf = io.BytesIO()
        fig.savefig(buf, format="png")
        buf.seek(0)
        images.append(Image.open(buf).copy())

        # Close buffer and figure
        buf.close()
        plt.close(fig)

    # Save the GIF with the specified FPS
    images[0].save(f"{log_name}.gif", save_all=True, append_images=images[1:], duration=duration, loop=0)


# Create a gif animation with a specific FPS
create_animation_gif(fps=30, end_idx=num_iterations, step=10)